Use previous lab notebooks to install and run a spark instance, and then follow the instructions within "TO DO" comments

In [ ]:
# TO DO: set-up spark

In [ ]:
# TO DO:  install findspark 

In [ ]:
# TO DO: init spark (ensure SPARK_HOME set to same version as we download)
import os


# use 'spark' as reference (as per standard approach)
spark = SparkSession.builder.appName("bikes").getOrCreate()

In [ ]:
# get bike hire file for given year from TfL open data
!wget https://cycling.data.tfl.gov.uk/usage-stats/cyclehireusagestats-2014.zip
!unzip cyclehireusagestats-2014.zip



In [ ]:
# TO DO: read a file into a DF called "j_df"


In [ ]:
# see how many entries (rows) in data
numRows = j_df.count()
print("there are ",numRows," rows")


In [ ]:
# how to do a histogram of e.g. duration
# there is no graphical element of Spark (CHECK) so need to do locally
# we can use Python libs pandas & matplotlib

# import Python libs
import matplotlib.pyplot as plt
import pandas as pd

plot_pdf = j_df[["Duration"]].toPandas() # take just "Duration" col, convert to pandas dataframe structure ready to plot
#print("we have ", plot_df.count()," elements to plot")
print("we use '_pdf' to denote 'pandas data frame'")

plot_pdf.plot(kind="hist")
plt.show()

The above graph is not very information so we need to wrangle our data...

In [ ]:
# recall...
j_df.agg({"Duration": "min"}).show()
j_df.agg({"Duration": "mean"}).show()
j_df.agg({"Duration": "max"}).show()
j_df.agg({"Duration": "skewness"}).show()

In [ ]:
# look at values (random sampling)
sample_df=j_df.sample(0.001) # 0.001 of 0.5M is 500
print("sample has ",sample_df.count()," elements")
plotSample_pdf = sample_df[["Bike Id", "Duration"]].toPandas()
plotSample_pdf.plot(kind="scatter", y="Duration", x="Bike Id") # scatter plot (duration .v. bikeID)
plt.show()

In [ ]:
# so vast majority of rides in this month have "Duration" under 2 hours
total = j_df.count()
numExceed = j_df.filter("Duration > 7200").count()
print("Of all rides, percentage over 2 hours:", (numExceed*100)/total)



In [ ]:
# DO DO: add a histogram of ride Duration less or equal to 2 hours

In [ ]:
# can refine further
# e.g. remove rides with Duration of 0 seconds or over 3600
rides_df = j_df.filter("Duration > 0").filter("Duration <= 3600")[["Duration"]]
# now convert to Pandas df and plot
rides_df.toPandas().plot(kind="hist")
plt.show()

In [ ]:
# ignore the top outliers
n = 50
threshold = j_df.orderBy("Duration", ascending=False).head(n)[n-1].Duration 
print("Ignoring over ", threshold, " seconds and less than 1 minute")

In [ ]:
# TO DO: manually use threshold for given month 
print("histogram with 100 bins and log of frequency for rides lasting more than 5 minutes")
j_df.filter("Duration > 300").filter("Duration < xxx")[["Duration"]].toPandas().plot(kind="hist", bins=100, log=True)
plt.show()

Comparing plots...

In [ ]:
# compare plots for jan and part of aug (file 9a)
print("comparing >=5 min but less than or equal to 2 hours by ploting PDF")
rides_df = j_df.filter("Duration >= 300").filter("Duration <= 7200")[["Duration"]]
# now convert to Pandas df and plot
rides_df.toPandas().plot(kind="hist", bins=23, density=True)
plt.xlabel("Duration / seconds")
plt.ylabel("PDF")
#plt.ylim((10.0,100000.0))
# get y range (so can use in next plot)
minY,maxY = plt.ylim()
plt.show()

aug_df = (spark.read.format("csv")
         .option("header", "true")
         .option("inferSchema", "true")
         .load('./9a*csv'))
print("for end aug dataset we have ", aug_df.count(), " rows of data")
aug_df.filter("Duration >= 300").filter("Duration < 7200")[["Duration"]].toPandas().plot(kind="hist", bins=23, density=True)
plt.xlabel("Duration / seconds")
plt.ylabel("PDF")
plt.ylim(minY, maxY) # ensure same scales for x- and y- axes
plt.show()
print("i.e. some shift towards longer rides?")


In [ ]:
# aug is covered in TWO files
# 8b (01Aug to 16Aug)
# 9a (17Aug to 31Aug)

# let's read in both and plot
aug_df = (spark.read.format("csv")
         .option("header", "true")
         .option("inferSchema", "true")
         .load(['./8b*csv', './9a*csv']))

print("ALL of aug has ", aug_df.count(), " entries")
# attempt to plot
aug_df.filter("Duration >= 300").filter("Duration < 7200")[["Duration"]].toPandas().plot(kind="hist", bins=23, density=True)
plt.xlabel("Duration / seconds")
plt.ylabel("PDF")
#plt.ylim(minY, maxY)
plt.show()

In [ ]:
# what about June (TWO files sort of)
june_df = (spark.read.format("csv")
         .option("header", "true")
         .option("inferSchema", "true")
         .load(['./6*csv',   # 25May to 21Jun
                './7*csv'])) # 22Jun to 19Jul
print("we have ", aug_df.count(), " entries")
# attempt to plot
june_df.filter("Duration >= 300").filter("Duration < 7200")[["Duration"]].toPandas().plot(kind="hist", bins=23, density=True)
plt.xlabel("Duration / seconds")
plt.ylabel("PDF")
#plt.ylim(minY, maxY)
plt.show()


let us look at UNION: read in each file then append 2nd DF to 1st DF



In [ ]:
# using 8b file and sampling
temp_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('./8b*csv')

# TO DO: how many rows in temp_df?

# sample (ratio to obtain) - NB this is not exact ratio
# let's take two samples and use UNION

sampA_df = temp_df.sample(0.00002)
print("sample A: ", sampA_df.count())
sampA_df.show(sampA_df.count())

sampB_df = temp_df.sample(0.00001)
print("sample B: ", sampB_df.count())
sampB_df.show(sampB_df.count())


In [ ]:
# now combine in to one dataframe (same schema (contrast 'join') but keeping all rows of data)
# this is a UNION
mix_df = sampA_df.union(sampB_df)
print("mix: ",mix_df.count())
mix_df.show(mix_df.count())

Example of some data wrangling

NB we expect an "analysis exception" when running the next cell

In [ ]:
# what about june?
# let's try reading datasets 6 and 7 then form their union
juneA_df = (spark.read.format("csv")
         .option("header", "true")
         .option("inferSchema", "true")
         .load('./6*csv'))  # 25May to 21Jun
juneB_df = (spark.read.format("csv")
         .option("header", "true")
         .option("inferSchema", "true")
         .load('./7*csv'))  # 22Jun to 19Jul

mix_df = juneA_df.union(juneB_df)
print("mix: ",mix_df.count())
mix_df.show(mix_df.count())

In [ ]:
# read the "Analysis Exception" carefully
# perhaps a clue?

# TO DO: print out schema + 2 rows for each juneA/B DF

# print scheme directly
juneA_df.printSchema()
juneB_df.printSchema()

In [ ]:
# so for some reason the '6' file (juneA dataframe) scheme is "incorrect"
# ... what can we do?

# first see if anything in these extra cols that we may need
juneA_df[["_c9", "_c10", "_c11"]].distinct().show()

In [ ]:
# so no actual info, thus we can amend our data set by ignoring these columns and using only "Duration" 
meld_df = juneA_df[["Duration"]].union(juneB_df[["Duration"]]) # i.e. a union of only the 'Duration' columns

# now filter the union, convert to Pandas and plot PDF
meld_df.filter("Duration >= 300").filter("Duration < 7200").toPandas().plot(kind="hist", bins=23, density=True)
plt.xlabel("Duration / seconds")
plt.ylabel("PDF")
#plt.ylim(minY, maxY)
plt.show()

Filter by timestamp (& using 'alias' for convenience)

In [ ]:
# finally we only want June (not May nor July which were included in the input files)

# conversion to expected timestamp format so can filter, and alias so easily refer to 'start'
temp_df = juneA_df.select(to_timestamp(col("Start Date"), format="dd/MM/yyyy HH:mm").alias("start"), "Duration")

# example how to filter for dates later than May, outputting example 5 dates for checking
# NB at this point temp_df still contains May & June
print("quick check we do not see any MAY dates")
temp_df.filter(temp_df.start > "2014-06-01 00:00:00").show(5)

# we can do similarly re juneB_df
temp2_df = juneB_df.select(to_timestamp(col('Start Date'), format="dd/MM/yyyy HH:mm").alias("start"), "Duration")

# do a UNION of only June data from the two datasets keeping only the Duration column
# i.e. we do a filter on each temp DF re 'start' then select only 'Duration' col and form the union of those subsets
june_df =  temp_df.filter(temp_df.start > "2014-06-01 00:00:00")[["Duration"]].union(temp2_df.filter(temp2_df.start < "2014-07-01 00:00:00")[["Duration"]])

# plot the June-only data for rides at least 5 mins but less than 2 hours (then compare to January, say)
print("plot PDF of cleansed data for June only, rides>=5m and <2hr")
june_df.filter("Duration >= 300").filter("Duration < 7200").toPandas().plot(kind="hist", bins=23, density=True)
plt.xlabel("Duration / seconds")
plt.ylabel("PDF")
#plt.ylim(minY, maxY)
plt.show()


In [ ]:
# TO DO: plot various months' PDF and gain empirical knowledge of how length of rides varies over the year 2014